In [56]:
#Библиотеки
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [57]:
#Удаление хтмл тэгов
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [59]:
#Извлечение цены
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [60]:
getPrice(flat_page)

7990000

In [ ]:
#Количество комнат
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    if room_number == '':
        room_number = '0'
    return float(room_number)

In [ ]:
#Расстояние от центра
def getDist(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    center_lat = 55.75370903771494 # координаты Красной площади
    center_lon = 37.61981338262558 
    dist = 111.2*((lat-center_lat)**2 + (lon-center_lon)**2)**0.5
    return dist

In [ ]:
# этаж, этажность, общ.пл., жил.пл., тип дома, тип жилья, балкон, телефон

def getTable(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    # этаж
    etaz = re.split('Этаж|Тип дома', table)[1]
    etaz = re.sub("[^0-9,/]", "", etaz)
    
    # этаж/этажность
    etaz_num = float(etaz.split("/")[0])
    if etaz.find('/') == -1:
        etaz_all= 0
    else:
        etaz_all = float(etaz.split("/")[1])
    
    # тип дома  и новизна
    htype = re.split('Тип дома|Тип продажи', table)[1]
    new_flat =0
    if htype.find('втор') != -1:
        new_flat = 1
    else:
        new_flat = 0
    house_type = 0
    if (htype.find('кирп') != -1) or (htype.find('монол') != -1) :
        house_type = 1
    else: house_type = 0
        
    # площадь
    tot_square = re.split('Общая площадь|\xa0м2', table)[1] 
    tot_square = tot_square.replace(',','.')
    if re.sub("[^0-9\.]", "", tot_square) == '':
        tot_square = 0
    else: 
        tot_square = float(re.sub("[^0123456789\.]", "", tot_square))
    
    #кухня
    kitchen = re.split('Площадь кухни|ануз', table)[1]
    if re.sub("[^0-9]", "", kitchen) == '' :
        kitchen = 0
    else:
        kitchen = re.sub(r'\s', '', kitchen) 
        kitchen = kitchen.split("м")[0]
        kitchen = re.sub("[^0-9,]", "", kitchen)
        kitchen = float(kitchen.replace(',','.'))
        
    # жилая площадь
    lsquare = re.split('Жилая площадь|Площадь кухни', table)[1]
    lsquare = re.sub(r'\s', '', lsquare) 
    lsquare = lsquare.split("м")[0]
    lsquare = re.sub("[^0-9,]", "", lsquare)
    lsquare
    if lsquare != '':
        lsquare = float(lsquare.replace(',','.'))
    else: lsquare = 0
    
    #балкон
    balcon = re.split('Балкон|Лифт', table)[1]
    balcon_flag = 0
    if (balcon.find('нет') != -1) or (balcon.find('–') != -1):
        balcon_flag = 0
    else:
        balcon_flag = 1
    
    #Телефон 
    phone = 0
    buff = ''
    if table.find('Телеф') == -1:
        phone =0
    else : 
        buff = re.split('Телефон|Вид', table)[1]
        if buff.find('да') != -1:
            phone =0
        else:
            phone = 1
    return etaz_all, etaz_num, new_flat, tot_square, balcon_flag, house_type, lsquare, kitchen, phone


In [ ]:
# извлечение ближайшего метро, времени до метро, пешком\транспорт

def getMetro(flat_page):
    metro = flat_page.find('div', attrs={'class':'object_descr_metro'})
    metro = html_stripper(metro)
    metro_name = ''
    for i in re.split('\n|,', metro):
        if ' ' in i:
            break
        else:
            metro_name += i
    metro_name = "".join(metro_name.split())
    metro_time =''
    for i in re.split(',|\n ', metro):
        if 'мин.' in i:
            break
        else:
            metro_time += i
    metro_time = "".join(metro_time.split())
    metro_time = re.sub("[^0-9]", "", metro_time)
    metro_time = metro_time.replace('1905','') # специально для метро 1905 года
    if metro_time == '':
        metro_time = '0'
    metro_walk = 0
    if metro.find('пешком') != -1:
        metro_walk = 1
    else:
        metro_walk = 0
    return float(metro_time), metro_walk

In [ ]:
#функция парсинга всего

def my_pars(flat_url):
    flat_page = requests.get(flat_url)
    flat_page = flat_page.content
    flat_page = BeautifulSoup(flat_page, 'lxml')	
    flatStats = {'District':0}  # заводим объект , округ = 0
    flatStats['Rooms'] = getRoom(flat_page)
    flatStats['Price'] = getPrice(flat_page)
    flatStats['Dist'] = getDist(flat_page)
    flatStats['Nfloors'], flatStats['Floor'], flatStats['New'], flatStats['Totsp'], flatStats['Bal'], flatStats['Brick'], flatStats['Livesp'], flatStats['Kitsp'], flatStats['Tel'] = getTable(flat_page)
    flatStats['Metrdist'], flatStats['Walk'] = getMetro(flat_page)
    flatStats['url'] = flat_url	
    return flatStats['Rooms'], flatStats['Price'], flatStats['Totsp'], flatStats['Livesp'], flatStats['Kitsp'], flatStats['Dist'], flatStats['Metrdist'], flatStats['Walk']	, flatStats['Brick'], flatStats['Tel'], flatStats['Bal'], flatStats['Floor'], flatStats['Nfloors'], flatStats['New'], flatStats['url']


In [ ]:
# Начало программы. Записываем все округа
districts=['','','','','','','','','','','','']

#ЦАО    
districts[0] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'    
#САО
districts[1] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#СВАО
districts[2] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ВАО
districts[3] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЮВАО
districts[4] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЮАО
districts[5] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЮЗАО
districts[6] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЗАО
districts[7] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#СЗАО
districts[8] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЗЕЛАО
districts[9] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=152&district%5B1%5D=153&district%5B2%5D=154&district%5B3%5D=355&district%5B4%5D=356&district%5B5%5D=357&district%5B6%5D=358&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#НАО
districts[10] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ТАО
districts[11] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district

In [ ]:
links = [] #массив для ссылок на квартиры
# Создадим датафрейм для 12 округов по 812 квартир в каждом

df = pd.DataFrame(np.random.randn(812*12, 15), columns=['Rooms','Price','Totsp','Livesp','Kitsp','Dist','Metrdist','Walk','Brick','Tel','Bal','Floor','Nfloors','New','URL'])


In [ ]:
#Два цикла, первый по - округам, второй - по квартирам
for i in range(0,12):
    for page in range(1, 30):
        page_url =  districts[i].format(page)
        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')
        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    for k in range(0,812):
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[k]) + '/'
        df.loc[k+i*812] = my_pars(flat_url)
    
df.to_csv("cian.csv")